In [ ]:
First, pip install pandas which we will be using to read and use our training data


In [96]:
# Create a virtual environment named 'venv'
!python -m venv venv

In [97]:
# Activate the virtual environment (this may vary depending on your operating system)
!source venv/bin/activate  # For Linux/macOS
# OR
#!venv\Scripts\activate  # For Windows


In [114]:
!pip install -q pandas


In [ ]:
Next need to import libraries to setup and get the environment ready

In [115]:
!pip install -q autogluon

In [116]:
from autogluon.tabular import TabularPredictor, TabularDataset
import pandas as pd

In [117]:
train_data = TabularDataset(data="train_car.csv")
test_data = TabularDataset(data="test_car.csv")

Loaded data from: train_car.csv | Columns = 4 / 4 | Rows = 20999 -> 20999
Loaded data from: test_car.csv | Columns = 3 / 3 | Rows = 1002 -> 1002


In [118]:
train_data.head()

,Name,Year,Miles,Price
0,Chevrolet Trax,2018,41946,16990
1,GMC Terrain,2020,45328,23990
2,Jeep Wrangler,2012,81068,21590
3,Jeep Renegade,2019,35372,21590
4,BMW X,20173,68992,22990


In [119]:
#sampling
sample_size = 1000
train_data_small = train_data.sample(n=sample_size, random_state=0)

train_data_small.head()

,Name,Year,Miles,Price
17021,Hyundai Elantra,2015,18824,18590
3259,MAZDA MX- Miata,20145,58700,19990
3833,Nissan Rogue,2020,60344,20106
3283,Mitsubishi Outlander Sport,2017,64442,15990
15517,Toyota Corolla,2014,84366,15590


In [120]:
predict_smaller = TabularPredictor(label="Price",
                                   eval_metric="mean_squared_error",
                                   path="predict_smaller"
                                  ).fit(train_data_small)

No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "predict_smaller"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.9.13
Operating System:   Darwin
Platform Machine:   x86_64
Platform Version:   Darwin Kernel Version 22.2.0: Fri Nov 11 02:03:51 PST 2022; root:xnu-8792.61.2~4/RELEASE_ARM64_T6000
CPU Count:          8
Memory Avail:       0.65 GB / 

In [121]:
predict_smaller.leaderboard(silent=True)

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-1.730569e+07,mean_squared_error,0.036233,35.299949,0.000317,0.179486,2,True,9
1,NeuralNetTorch,-1.930496e+07,mean_squared_error,0.012946,31.590965,0.012946,31.590965,1,True,8
2,CatBoost,-1.974400e+07,mean_squared_error,0.001857,1.267805,0.001857,1.267805,1,True,4
3,NeuralNetFastAI,-2.000072e+07,mean_squared_error,0.021113,2.261693,0.021113,2.261693,1,True,6
4,XGBoost,-2.045965e+07,mean_squared_error,0.009790,8.573687,0.009790,8.573687,1,True,7
5,ExtraTreesMSE,-2.406562e+07,mean_squared_error,0.040580,0.275830,0.040580,0.275830,1,True,5
6,RandomForestMSE,-2.834276e+07,mean_squared_error,0.039809,0.336203,0.039809,0.336203,1,True,3
7,KNeighborsUnif,-3.583323e+07,mean_squared_error,0.015272,0.016900,0.015272,0.016900,1,True,1
8,KNeighborsDist,-3.747217e+07,mean_squared_error,0.016966,0.005337,0.016966,0.005337,1,True,2


In [122]:
test_data.head()

,Name,Year,Miles
0,Toyota Camry,2019,19796
1,Kia Niro,2019,27189
2,Hyundai Elantra,2015,18824
3,Honda CR-V,2014,64396
4,Ford Explorer,2015,105432


In [123]:
price_predict = predict_smaller.predict(test_data)
print(f"Some predicted results") 
print(f"{price_predict[0:10]}")

Some predicted results
0    24084.322266
1    24642.746094
2    18707.195312
3    19623.396484
4    17540.898438
5    16146.230469
6    15496.059570
7    21057.402344
8    23260.027344
9    17788.533203
Name: Price, dtype: float32


In [124]:
output_data = pd.DataFrame(columns=["Name","Price"])
output_data["Name"] = test_data["Name"].tolist()
print(f"{output_data[0:10]}")

               Name Price
0      Toyota Camry   NaN
1          Kia Niro   NaN
2   Hyundai Elantra   NaN
3        Honda CR-V   NaN
4     Ford Explorer   NaN
5     Nissan Sentra   NaN
6   Hyundai Elantra   NaN
7      Nissan Rogue   NaN
8      Jeep Compass   NaN
9   Chevrolet Cruze   NaN


In [125]:
output_data["Price"] = price_predict
print(f"{output_data[0:10]}")

               Name         Price
0      Toyota Camry  24084.322266
1          Kia Niro  24642.746094
2   Hyundai Elantra  18707.195312
3        Honda CR-V  19623.396484
4     Ford Explorer  17540.898438
5     Nissan Sentra  16146.230469
6   Hyundai Elantra  15496.059570
7      Nissan Rogue  21057.402344
8      Jeep Compass  23260.027344
9   Chevrolet Cruze  17788.533203


In [126]:
actual_data = TabularDataset(data="test_actual.csv")
print(f"{actual_data[0:10]}")

Loaded data from: test_actual.csv | Columns = 4 / 4 | Rows = 1002 -> 1002


               Name  Year   Miles  Price
0      Toyota Camry  2019   19796  24590
1          Kia Niro  2019   27189  24990
2   Hyundai Elantra  2015   18824  18590
3        Honda CR-V  2014   64396  19590
4     Ford Explorer  2015  105432  17590
5     Nissan Sentra  2017   59223  14990
6   Hyundai Elantra  2012   67040  13990
7      Nissan Rogue  2018   34701  20990
8      Jeep Compass  2019   21544  23590
9   Chevrolet Cruze  2017   39976  17990


In [127]:
output_data.to_csv("output_smaller.csv", index=False)

In [128]:
#2 create model
predictor = TabularPredictor(label="Price",
                             eval_metric="mean_squared_error",
                             path="predictor"
                            ).fit(train_data=train_data,
                                 time_limit=5*60)

No path specified. Models will be saved in: "AutogluonModels/ag-20240207_025511"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ... Time limit = 300s
AutoGluon will save models to "AutogluonModels/ag-20240207_025511"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.9.13
Operating System:   Darwin
Platform Machine:   x86_64
Platform Version:   Darwin Kernel Version 22.2.0: Fri N

In [129]:
#3 predict using predictor model
final_prediction = predictor.predict(test_data)

#4 output save
output_data = pd.DataFrame(columns=["Name","Price"])
output_data["Name"] = test_data["Name"].tolist()
output_data["Price"] = final_prediction

print(f"{output_data[0:10]}")

               Name  Price
0      Toyota Camry  24590
1          Kia Niro  24990
2   Hyundai Elantra  18590
3        Honda CR-V  19590
4     Ford Explorer  17590
5     Nissan Sentra  14990
6   Hyundai Elantra  13990
7      Nissan Rogue  20990
8      Jeep Compass  23590
9   Chevrolet Cruze  17990


In [130]:
actual_data = TabularDataset(data="test_actual.csv")
print(f"{actual_data[0:10]}")

Loaded data from: test_actual.csv | Columns = 4 / 4 | Rows = 1002 -> 1002


               Name  Year   Miles  Price
0      Toyota Camry  2019   19796  24590
1          Kia Niro  2019   27189  24990
2   Hyundai Elantra  2015   18824  18590
3        Honda CR-V  2014   64396  19590
4     Ford Explorer  2015  105432  17590
5     Nissan Sentra  2017   59223  14990
6   Hyundai Elantra  2012   67040  13990
7      Nissan Rogue  2018   34701  20990
8      Jeep Compass  2019   21544  23590
9   Chevrolet Cruze  2017   39976  17990


In [131]:
output_data.to_csv("output.csv", sep='\t', encoding = 'utf-8')

In [132]:
predict_smaller.feature_importance(train_data_small)

Computing feature importance via permutation shuffling for 3 features using 1000 rows with 5 shuffle sets...
	24.17s	= Expected runtime (4.83s per shuffle set)
	7.57s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
Name,2.328821e+07,7.049924e+05,1.006571e-07,5,2.473979e+07,2.183662e+07
Miles,1.846711e+07,1.135868e+06,1.708871e-06,5,2.080588e+07,1.612834e+07
Year,7.906518e+06,4.785839e+05,1.603075e-06,5,8.891929e+06,6.921107e+06


In [133]:
predictor.feature_importance(train_data)

Computing feature importance via permutation shuffling for 3 features using 5000 rows with 5 shuffle sets...
	36.68s	= Expected runtime (7.34s per shuffle set)
	25.72s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
Miles,10432.54276,122.123786,2.252902e-09,5,10683.997323,10181.088197
Year,6450.96520,86.952688,3.960129e-09,5,6630.001987,6271.928413
Name,2741.22848,351.781874,3.184326e-05,5,3465.552220,2016.904740
